In [1]:
#Variabilil globali
width = 256
height = 256

# TODO:
- Far partire Visdom
- Far funzionare accuracy.
- Sistemare cartelle e codice.
- Data agumentation
- Finetuning -->(Ipotesi utilizzare alexnet implementata senza classificazione)
- Aumentare il knn con data agumentation.

# Import necessari

In [2]:
import torchvision.models as models
from torch.utils.data.dataset import Dataset
from PIL import Image
from os import path
from torchvision import transforms
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier as KNN
from torch.autograd import Variable


# Classe per la gestione del dataset

In [3]:
np.random.seed(1234)
torch.random.manual_seed(1234);

class ScenesDataset(Dataset):
    def __init__(self,base_path,txt_list,transform=None):
        #conserviamo il path alla cartella contenente le immagini
        self.base_path=base_path
        #carichiamo la lista dei file
        #sarà una matrice con n righe (numero di immagini) e 2 colonne (path, etichetta)
        self.images = np.loadtxt(txt_list,dtype=str,delimiter=',')
        #print("self.images ha i seguenti elementi:", len(self.images))
        #conserviamo il riferimento alla trasformazione da applicare
        self.transform = transform
    def __getitem__(self, index):
        #print("Get item numero -->", index)
        #recuperiamo il path dell'immagine di indice index e la relativa etichetta
        f,c = self.images[index]
        #carichiamo l'immagine utilizzando PIL e facciamo il resize a 3 canali.
        im = Image.open(path.join(self.base_path, f)).convert("RGB")
        
        #Resize:
        im = im.resize((width,height))
        #se la trasfromazione è definita, applichiamola all'immagine
        if self.transform is not None:
            im = self.transform(im)       
        
        #convertiamo l'etichetta in un intero
        label = int(c)
        #restituiamo un dizionario contenente immagine etichetta
        #print("Mentre creo il tutto, label vale-->", label, ", name vale -->", f)
        return {'image' : im, 'label':label, 'name': f}
    #restituisce il numero di campioni: la lunghezza della lista "images"
    def __len__(self):
        #print("Ho invocato len, vale-->", len(self.images))
        return len(self.images)

# Prendiamo media e varianza per normalizzare successivamente i dati

In [4]:
dataset = ScenesDataset('Dataset','train.txt',transform=transforms.ToTensor())
m = np.zeros(3)
for sample in dataset:
    m+= np.array(sample['image'].sum(1).sum(1)) #accumuliamo la somma dei pixel canale per canale
#dividiamo per il numero di immagini moltiplicato per il numero di pixel
m=m/(len(dataset)*width*height)
#procedura simile per calcolare la deviazione standard
s = np.zeros(3)
for sample in dataset:
    s+= np.array(((sample['image']-torch.Tensor(m).view(3,1,1))**2).sum(1).sum(1))
s=np.sqrt(s/(len(dataset)*width*height))
print("Medie",m)
print("Dev.Std.",s)

Medie [0.53432863 0.50716533 0.5028028 ]
Dev.Std. [0.35351087 0.32069801 0.2918144 ]


# Creiamo Oggetto Dataset e dataloader

In [5]:
transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(m,s)])
barilla_train = ScenesDataset('Dataset','train.txt',transform=transformss)
barilla_test = ScenesDataset('Dataset','test.txt',transform=transformss)
barilla_train_loader = torch.utils.data.DataLoader(barilla_train, batch_size=1, num_workers=0, shuffle=True)
barilla_test_loader = torch.utils.data.DataLoader(barilla_test, batch_size=1, num_workers=0)

# Utilizziamo AlexNet con i parametri pre-allenati ed eliminiamo il layer di classificazione.

In [6]:
net = models.alexnet(pretrained=True)
classifier = nn.Sequential(*list(net.classifier.children())[:-1])
net.classifier = classifier
sum([p.numel() for p in net.parameters()])

57003840

# Funzione per calcolare l'accuracy.

In [12]:
# Prende in input l'array di feature e il classificatore(knn.) 
def accuracy(classifier, samples):
    right_pred = 0
    for i in range(len(samples)):
        pred_label = classifier.predict(samples[i]["feature"].cpu().detach().numpy().reshape(1, -1))
        if pred_label[0] == samples[i]["label"]:
            right_pred += 1
            
    return float(right_pred)/len(samples)

# Funzione per estrazione feature:
- In input dataset (dataloader) e rete.

In [8]:
def extract_features(dataset, net):
    #Presa ogni riga del dataloader li passa alla net senza attivare il layer di classificazione
    feature_dataset = []
    print("Avviato extract_feature.")
    for i, dataset_train in enumerate(dataset):
        x=Variable(dataset_train['image'], requires_grad=False)
        y=Variable(dataset_train['label'])
        x, y = x.cpu(), y.cpu()
        if torch.cuda.is_available():
            x, y = x.cuda(), y.cuda()
            #print("Con cuda")
        output = net(x)
        feature_dataset.append({"label": dataset_train['label'], "feature":output, "name": dataset_train['name']})
    return feature_dataset

# Funzione per creare l'input per l'oggetto dataframe:
- In input dataset (dataloader) e rete.

In [9]:
def get_dataframe(dataset, net):
    print("Avviato get_dataframe.")
    feature_dataset = extract_features(dataset, net)  
    feature_dataset_matrix = np.zeros((len(feature_dataset), len(feature_dataset[0]["feature"][0])))    
    #Qui abbiamo nelle righe tutte le immagini, nella lable feature tutte le 9000 colonne, ossia le feature.
    label_array = np.zeros(len(feature_dataset))
    for i in range(0, len(feature_dataset)):#302
        for j in range(0, len(feature_dataset[0]["feature"][0])):#9206
            if j == 0:#salviamo la y finale nell'array label_array
                label_array[i] = feature_dataset[i]['label'][0]
                print(i, end= " ")
            feature_dataset_matrix[i][j] =feature_dataset[i]["feature"][0][j] 

    return feature_dataset_matrix, label_array

# Creiamo il dataframe e il knn

In [10]:
knn_1 = KNN(n_neighbors=1)

if torch.cuda.is_available():
    net = net.cuda()
    torch.cuda.empty_cache()
net.eval()

input_for_datafram_train, label_array_train = get_dataframe(barilla_train_loader, net)
df = pd.DataFrame(input_for_datafram_train)
knn_1.fit(df, label_array_train)

Avviato get_dataframe.
Avviato extract_feature.
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 26

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [13]:
feature_test = extract_features(barilla_test_loader, net)
print(accuracy(knn_1, feature_test))

Avviato extract_feature.
0.506578947368421
